In [2]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

In [3]:
# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf

In [4]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [5]:
# Keras Specific
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

In [6]:
import pandas as pd

url = "https://dataclass-project4.s3.amazonaws.com/fer2013.csv"
df = pd.read_csv(url)

In [7]:
# df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
# Turning the pixels column into a list of arrays, so each image is an array
X_values = df['pixels'].tolist()
X = []
for i in X_values:
  X_list = [int(x) for x in i.split(' ')]
  X_list = np.asarray(X_list)
  X.append(X_list)

In [9]:
X = np.asarray(X)

In [10]:
y_df = pd.get_dummies(df['emotion'])
y_df.head()

y = []
for index, row in y_df.iterrows():
    row_array = np.array(row.values)
    y.append(row_array)
y = np.asarray(y)    

In [11]:
# Splitting the data into training and testing sets
# Before the model creation, you need to reshape X
# Assuming that you have grayscale images of size 48x48
X = X.reshape(-1, 48, 48, 1)  # reshaping into (num_images, 48, 48, 1)

# Now split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert lists to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Rescale the images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255


In [12]:
datagen = ImageDataGenerator(
    rotation_range=20,       # Rotate images randomly within the range [-20, 20] degrees
    width_shift_range=0.2,   # Shift the width of the images randomly by up to 20% of the total width
    height_shift_range=0.2,  # Shift the height of the images randomly by up to 20% of the total height
    shear_range=0.2,         # Apply shear transformations randomly within the range [-0.2, 0.2]
    zoom_range=0.2,          # Randomly zoom into images by up to 20%
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest'      # Fill any newly created pixels after rotation or width/height shift
)

In [13]:
# Create a generator for training data
train_generator = datagen.flow(X_train_scaled, y_train, batch_size=32)
# Create a generator for validation data
validation_generator = datagen.flow(X_test_scaled, y_test, batch_size=32)

In [15]:
# Creating the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Activation, Flatten

model = Sequential()

model.add(Conv2D(64,kernel_size=(3, 3), activation='relu', input_shape=(48,48,1), data_format="channels_last"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64*2,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64*2,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64*4,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64*4,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64*8,kernel_size=(3, 3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64*8,kernel_size=(3, 3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(8*64, activation='elu'))
model.add(Dropout(0.4))
model.add(Dense(4*64, activation='elu'))
model.add(Dropout(0.4))
model.add(Dense(2*64, activation='elu'))
model.add(Dropout(0.4))

#output layer
model.add(Dense(units=7, activation='softmax'))

In [ ]:
# model.summary()

In [16]:
# Compiling the model
model.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [14]:
# Fitting the model
model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator
    shuffle=True,
    verbose=2
)

Epoch 1/100
898/898 - 23s - loss: 1.9634 - accuracy: 0.3228 - 23s/epoch - 26ms/step
Epoch 2/100
898/898 - 10s - loss: 1.4846 - accuracy: 0.4403 - 10s/epoch - 11ms/step
Epoch 3/100
898/898 - 10s - loss: 1.3580 - accuracy: 0.4833 - 10s/epoch - 12ms/step
Epoch 4/100
898/898 - 11s - loss: 1.2859 - accuracy: 0.5125 - 11s/epoch - 12ms/step
Epoch 5/100
898/898 - 12s - loss: 1.2392 - accuracy: 0.5331 - 12s/epoch - 13ms/step
Epoch 6/100
898/898 - 11s - loss: 1.2028 - accuracy: 0.5448 - 11s/epoch - 12ms/step
Epoch 7/100
898/898 - 10s - loss: 1.1615 - accuracy: 0.5618 - 10s/epoch - 11ms/step
Epoch 8/100
898/898 - 10s - loss: 1.1220 - accuracy: 0.5769 - 10s/epoch - 11ms/step
Epoch 9/100
898/898 - 10s - loss: 1.0904 - accuracy: 0.5908 - 10s/epoch - 11ms/step
Epoch 10/100
898/898 - 10s - loss: 1.0642 - accuracy: 0.5975 - 10s/epoch - 11ms/step
Epoch 11/100
898/898 - 10s - loss: 1.0349 - accuracy: 0.6136 - 10s/epoch - 11ms/step
Epoch 12/100
898/898 - 10s - loss: 1.0047 - accuracy: 0.6232 - 10s/epoch -

In [1]:
# Evaluating the model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

# 1st trial - Loss: 1.9519965648651123, Accuracy: 0.6375035047531128
# Dropout 20% on each layer
# 2nd trial - Loss: 1.2198774814605713, Accuracy: 0.6517135500907898
# Dropout 40% on each layer
# 3rd trial - Loss: 1.1788020133972168, Accuracy: 0.6213430166244507
# Added three dense layers
# 4th trial - Loss: 1.1446014642715454, Accuracy: 0.5862357020378113
# Changed activation functions from relu to tanh
# 5th trial - Loss: 1.1010137796401978, Accuracy: 0.6258010864257812
# Changed activation functions from tanh to elu
# 6th trial - Loss: 1.3327525854110718, Accuracy: 0.6281694173812866
# Removed dense layers and changed convolution layers from relu to elu
# 7th trial - Loss: 1.3180416822433472, Accuracy: 0.6278907656669617
# Changed to dropout to 50% on each layer, slow learning rate
# 8th trial - 
# Using Data Augmentation, returned learning rate to default.

NameError: ignored

In [16]:
# Saving the model
model.save("emotion_model.h5")

In [17]:
# Loading the model
from tensorflow.keras.models import load_model
emotion_model = load_model('emotion_model.h5')

In [18]:
# Making predictions
predicted = emotion_model.predict(X_test_scaled)
predicted = np.argmax(predicted, axis=1)
predicted

225/225 [==============================] - 1s 3ms/step


array([3, 3, 0, ..., 0, 2, 2])

In [19]:
# Creating a dataframe of the predictions
predicted_df = pd.DataFrame(predicted)
predicted_df = predicted_df.rename(columns={0: "Predicted"})
predicted_df.head()

,Predicted
0,3
1,3
2,0
3,4
4,6


In [20]:
# Creating a dataframe of the actual values
y_test_actual = np.argmax(y_test, axis=1)

# Creating a dataframe of the actual values
y_test_df = pd.DataFrame(y_test_actual, columns=["Actual"])

y_test_df.head()

,Actual
0,3
1,6
2,0
3,4
4,0


In [21]:
# Merging the two dataframes
merged_df = pd.merge(y_test_df, predicted_df, left_index=True, right_index=True)
merged_df.head()

,Actual,Predicted
0,3,3
1,6,3
2,0,0
3,4,4
4,0,6


In [22]:
# Creating a dataframe of the emotions
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotions_df = pd.DataFrame(emotions)
emotions_df = emotions_df.rename(columns={0: "Emotions"})
emotions_df.head()

,Emotions
0,Angry
1,Disgust
2,Fear
3,Happy
4,Sad


In [23]:
# Merging the emotions dataframe with the merged dataframe
merged_df = pd.merge(merged_df, emotions_df, left_on='Actual', right_index=True)
merged_df.head()

,Actual,Predicted,Emotions
0,3,3,Happy
11,3,3,Happy
25,3,5,Happy
26,3,3,Happy
31,3,3,Happy


In [24]:
# Merging the emotions dataframe with the merged dataframe
merged_df = pd.merge(merged_df, emotions_df, left_on='Predicted', right_index=True)
merged_df.head()

,Actual,Predicted,Emotions_x,Emotions_y
0,3,3,Happy,Happy
11,3,3,Happy,Happy
26,3,3,Happy,Happy
31,3,3,Happy,Happy
32,3,3,Happy,Happy
